In [1]:
# Check if the easydiffraction library is installed.
# If not, install it including the 'visualization' extras.
# This is needed, e.g., when running this as a notebook via Google Colab or
# Jupyter Notebook in an environment where the library is not pre-installed.
import builtins
import importlib.util

if hasattr(builtins, '__IPYTHON__'):
    if importlib.util.find_spec('easydiffraction') is None:
        !pip install 'easydiffraction[visualization]'


# Structure Refinement: LBCO+Si, McStas

This example demonstrates a Rietveld refinement of La0.5Ba0.5CoO3 crystal
structure with a small amount of Si phase using time-of-flight neutron powder
diffraction data simulated with McStas.

## Import Library

In [2]:
from easydiffraction import Experiment
from easydiffraction import Project
from easydiffraction import SampleModel
from easydiffraction import download_from_repository

## Define Sample Models

This section shows how to add sample models and modify their parameters.

### Create Sample Model 1: LBCO

In [3]:
model_1 = SampleModel('lbco')

#### Set Space Group

In [4]:
model_1.space_group.name_h_m = 'P m -3 m'
model_1.space_group.it_coordinate_system_code = '1'

#### Set Unit Cell

In [5]:
model_1.cell.length_a = 3.8909

#### Set Atom Sites

In [6]:
model_1.atom_sites.add(
    'La',
    'La',
    0,
    0,
    0,
    wyckoff_letter='a',
    b_iso=0.2,
    occupancy=0.5,
)
model_1.atom_sites.add(
    'Ba',
    'Ba',
    0,
    0,
    0,
    wyckoff_letter='a',
    b_iso=0.2,
    occupancy=0.5,
)
model_1.atom_sites.add(
    'Co',
    'Co',
    0.5,
    0.5,
    0.5,
    wyckoff_letter='b',
    b_iso=0.2567,
)
model_1.atom_sites.add(
    'O',
    'O',
    0,
    0.5,
    0.5,
    wyckoff_letter='c',
    b_iso=1.4041,
)

### Create Sample Model 2: Si

In [7]:
model_2 = SampleModel('si')

#### Set Space Group

In [8]:
model_2.space_group.name_h_m = 'F d -3 m'
model_2.space_group.it_coordinate_system_code = '2'

#### Set Unit Cell

In [9]:
model_2.cell.length_a = 5.43146

#### Set Atom Sites

In [10]:
model_2.atom_sites.add(
    'Si',
    'Si',
    0.0,
    0.0,
    0.0,
    wyckoff_letter='a',
    b_iso=0.0,
)

## Define Experiment

This section shows how to add experiments, configure their parameters, and
link the sample models defined in the previous step.

#### Download Data

In [11]:
download_from_repository('mcstas_lbco-si.xye', destination='data')


⚠️ Warning
File 'data/mcstas_lbco-si.xye' already exists and will not be overwritten.


#### Create Experiment

In [12]:
experiment = Experiment(
    'mcstas',
    sample_form='powder',
    beam_mode='time-of-flight',
    radiation_probe='neutron',
    scattering_type='bragg',
    data_path='data/mcstas_lbco-si.xye',
)


Data loaded successfully
Experiment 🔬 'mcstas'. Number of data points: 1000


#### Set Instrument

In [13]:
experiment.instrument.setup_twotheta_bank = 94.90931761529106
experiment.instrument.calib_d_to_tof_offset = 0.0
experiment.instrument.calib_d_to_tof_linear = 58724.76869981215
experiment.instrument.calib_d_to_tof_quad = -0.00001

#### Set Peak Profile

In [14]:
# experiment.peak_profile_type = 'pseudo-voigt * ikeda-carpenter'
experiment.peak.broad_gauss_sigma_0 = 45137
experiment.peak.broad_gauss_sigma_1 = -52394
experiment.peak.broad_gauss_sigma_2 = 22998
experiment.peak.broad_mix_beta_0 = 0.0055
experiment.peak.broad_mix_beta_1 = 0.0041
experiment.peak.asym_alpha_0 = 0
experiment.peak.asym_alpha_1 = 0.0097

#### Set Background

Select the background type.

In [15]:
experiment.background_type = 'line-segment'


Background type for experiment 'mcstas' changed to
line-segment


Add background points.

In [16]:
experiment.background.add(x=45000, y=0.2)
experiment.background.add(x=50000, y=0.2)
experiment.background.add(x=55000, y=0.2)
experiment.background.add(x=65000, y=0.2)
experiment.background.add(x=70000, y=0.2)
experiment.background.add(x=75000, y=0.2)
experiment.background.add(x=80000, y=0.2)
experiment.background.add(x=85000, y=0.2)
experiment.background.add(x=90000, y=0.2)
experiment.background.add(x=95000, y=0.2)
experiment.background.add(x=100000, y=0.2)
experiment.background.add(x=105000, y=0.2)
experiment.background.add(x=110000, y=0.2)

#### Set Linked Phases

In [17]:
experiment.linked_phases.add('lbco', scale=4.0)
experiment.linked_phases.add('si', scale=0.2)

## Define Project

The project object is used to manage sample models, experiments, and analysis.

#### Create Project

In [18]:
project = Project()

#### Set Plotting Engine

In [19]:
project.plotter.engine = 'plotly'


Current plotter changed to
plotly


#### Add Sample Models

In [20]:
project.sample_models.add(model_1)
project.sample_models.add(model_2)

#### Show Sample Models

In [21]:
project.sample_models.show_names()


Defined sample models 🧩
['lbco', 'si']


#### Add Experiments

In [22]:
project.experiments.add(experiment)

#### Set Excluded Regions

Show measured data as loaded from the file.

In [23]:
project.plot_meas(expt_name='mcstas')

Add excluded regions.

In [24]:
experiment.excluded_regions.add(start=0, end=40000)
experiment.excluded_regions.add(start=108000, end=200000)

Show excluded regions.

In [25]:
experiment.excluded_regions.show()


Excluded regions


start,end
0,40000
108000,200000


Show measured data after adding excluded regions.

In [26]:
project.plot_meas(expt_name='mcstas')

Show experiment as CIF.

In [27]:
project.experiments['mcstas'].show_as_cif()


Experiment 🔬 'mcstas' as cif


data_mcstas
""
_expt_type.beam_mode time-of-flight
_expt_type.radiation_probe neutron
_expt_type.sample_form powder
_expt_type.scattering_type bragg
""
_instr.d_to_tof_linear 58724.76869981215
_instr.d_to_tof_offset 0.0
_instr.d_to_tof_quad -1e-05
_instr.d_to_tof_recip 0.0


## Perform Analysis

This section outlines the analysis process, including how to configure
calculation and fitting engines.

#### Set Calculator

In [28]:
project.analysis.current_calculator = 'cryspy'


Current calculator changed to
cryspy


#### Set Minimizer

In [29]:
project.analysis.current_minimizer = 'lmfit (leastsq)'


Current minimizer changed to
lmfit (leastsq)


#### Set Fitting Parameters

Set sample model parameters to be optimized.

In [30]:
model_1.cell.length_a.free = True
model_1.atom_sites['Co'].b_iso.free = True
model_1.atom_sites['O'].b_iso.free = True

model_2.cell.length_a.free = True

Set experiment parameters to be optimized.

In [31]:
experiment.linked_phases['lbco'].scale.free = True
experiment.linked_phases['si'].scale.free = True

experiment.peak.broad_gauss_sigma_0.free = True
experiment.peak.broad_gauss_sigma_1.free = True
experiment.peak.broad_gauss_sigma_2.free = True

experiment.peak.asym_alpha_1.free = True
experiment.peak.broad_mix_beta_0.free = True
experiment.peak.broad_mix_beta_1.free = True

for point in experiment.background:
    point.y.free = True

#### Perform Fit

In [32]:
project.analysis.fit()


Using experiment 🔬 'mcstas' for 'single' fitting
🚀 Starting fit process with 'lmfit (leastsq)'...
📈 Goodness-of-fit (reduced χ²) change:


iteration,χ²,improvement [%]
1,301.41,
29,3.36,98.9% ↓
55,3.24,3.7% ↓
134,3.24,


🏆 Best goodness-of-fit (reduced χ²) is 3.24 at iteration 133
✅ Fitting complete.



Fit results
✅ Success: True
⏱️ Fitting time: 2.83 seconds
📏 Goodness-of-fit (reduced χ²): 3.24
📏 R-factor (Rf): 4.76%
📏 R-factor squared (Rf²): 5.16%
📏 Weighted R-factor (wR): 5.22%
📈 Fitted parameters:


,datablock,category,entry,parameter,start,fitted,uncertainty,units,change
1,lbco,atom_sites,Co,b_iso,0.2567,0.3233,0.0853,Å²,25.94 % ↑
2,lbco,atom_sites,O,b_iso,1.4041,2.1686,0.0282,Å²,54.45 % ↑
3,lbco,cell,,length_a,3.8909,3.8904,0.0001,Å,0.01 % ↓
4,si,cell,,length_a,5.4315,5.4327,0.0006,Å,0.02 % ↑
5,mcstas,background,45000,y,0.2000,0.2502,0.0082,,25.09 % ↑
6,mcstas,background,50000,y,0.2000,0.2498,0.0080,,24.91 % ↑
7,mcstas,background,55000,y,0.2000,0.2674,0.0047,,33.68 % ↑
8,mcstas,background,65000,y,0.2000,0.2551,0.0036,,27.53 % ↑
9,mcstas,background,70000,y,0.2000,0.2463,0.0035,,23.17 % ↑
10,mcstas,background,75000,y,0.2000,0.2417,0.0027,,20.86 % ↑


#### Plot Measured vs Calculated

In [33]:
project.plot_meas_vs_calc(expt_name='mcstas')